In [1]:
import boto3
import os
from os import path

In [2]:
from dotenv import load_dotenv
load_dotenv()
MY_ENV_VAR = os.getenv('DB_DESTINATION_HOST')
print(MY_ENV_VAR)

rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net


In [3]:
load_dotenv()
dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_db = os.environ.get('DB_DESTINATION_NAME')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')

In [4]:
bucket_name = os.environ.get('S3_BUCKET_NAME')
print(bucket_name)

s3-student-mle-20240226-38fc211d67


In [5]:
session = boto3.session.Session()

ENDPOINT = "https://storage.yandexcloud.net"

session = boto3.Session(
                        aws_access_key_id= (os.environ['AWS_ACCESS_KEY_ID']),
                        aws_secret_access_key= (os.environ['AWS_SECRET_ACCESS_KEY']),
                        region_name="ru-central1"
    )

s3 = session.client(
    "s3", endpoint_url=ENDPOINT)

Создание ПАПКИ в S3

In [6]:
#response = s3. put_object(
#                          Bucket=bucket_name,
#                          Body='',
#                          Key='9/2817ee4a2b78485ba867b214a132e169/artifacts/models/'
                          )

Запись файла в s3

In [7]:
# Upload the file to S3
#s3.upload_file('/home/mle-user/mlflow/mle-mlflow/users_churn.csv', 
#               bucket_name, 
#               '9/2817ee4a2b78485ba867b214a132e169/artifacts/models/users_churn.csv')

Стирание файлов в S3

In [22]:
for key in s3.list_objects(Bucket=os.environ.get('S3_BUCKET_NAME'))['Contents']:
    print(key['Key'])
    #s3.delete_object(Bucket=bucket_name, Key=key['Key'])

KeyError: 'Contents'

In [21]:
#bucket_name = 's3-student-mle-20240226-38fc211d67'
#bucket_name = dst_password = os.environ.get('S3_BUCKET_NAME')
for key in s3.list_objects(Bucket=os.environ.get('S3_BUCKET_NAME'))['Contents']:
    print(key['Key'])

KeyError: 'Contents'

ЗАГРУЗКА данных из ПОСТГРЕСС

In [10]:
import psycopg2 as psycopg
import boto, boto3
import pandas as pd
import os, importlib
import mlflow

import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [11]:
postgres_credentials = {
    "host":     (os.environ['DB_DESTINATION_HOST']),
    "port":     (os.environ['DB_DESTINATION_PORT']),
    "dbname":   (os.environ['DB_DESTINATION_NAME']),
    "user":     (os.environ['DB_DESTINATION_USER']),
    "password": (os.environ['DB_DESTINATION_PASSWORD'])
}
print(postgres_credentials)

{'host': 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', 'port': '6432', 'dbname': 'playground_mle_20240226_38fc211d67', 'user': 'mle_20240226_38fc211d67', 'password': '3639df44fa6c4adf8f058988c14bee9a'}


In [12]:
# определяем название таблицы, в которой хранятся наши данные
TABLE_NAME = 'users_churn'
#connection = {"sslmode": "verify-full", "target_session_attrs": "read-write"}
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])
connection.update(postgres_credentials)

In [13]:
# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций с базой данных
# причём закрыто оно будет даже в случае ошибки при работе с базой данных
# это нужно, чтобы не допустить так называемую "утечку памяти"
with psycopg.connect(**connection) as conn:
# создаём объект курсора для выполнения запросов к базе данных 
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
				
				# извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

				# получаем список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

In [14]:
# создаём объект DataFrame из полученных данных и имён столбцов 
# это позволяет удобно работать с данными в Python с использованием библиотеки Pandas
data = pd.DataFrame(data, columns=columns)

In [15]:
data.head(3)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,11,9763-GRSKD,2019-01-01,NaT,Month-to-month,Yes,Mailed check,49.95,587.45,DSL,...,No,No,No,No,Male,0,Yes,Yes,No,0
1,12,7469-LKBCI,2018-10-01,NaT,Two year,No,Credit card (automatic),18.95,326.80,None,...,None,None,None,None,Male,0,No,No,No,0
2,13,8091-TTVAX,2015-04-01,NaT,One year,No,Credit card (automatic),100.35,5681.10,Fiber optic,...,Yes,No,Yes,Yes,Male,0,Yes,No,Yes,0
